In [12]:
# Importation des packaques
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [3]:
# Chargement du fichier vgsales
vgsales = pd.read_csv('vgsales.csv')

#vgsales['Year']=vgsales['Year'].fillna(0).astype(int)
vgsales.isna().sum() #==> there is 58 Publisher : NA 
vgsales.head()

# I create an other df with 3 rows to test the code below.
vg=vgsales[0:100]
vg.head()


vgsales = pd.read_csv('vgsales_full/vgsales_full.csv')
vgsales['Year']=vgsales['Year'].fillna(0).astype(int).astype(str)
# No need to create  littlest test set
vg=vgsales
#vg['description'][1]



In [54]:
# Importation du fichier de scraping
scraping = pd.read_csv('vgsales_scraping.csv')
#print(scraping.isnull().sum())

# Scraping Metascore
# On recherche les nombres dans Metascore correspond respectivement à la note et au nombre de vote
r = re.compile(r"[0-9]+")

# On crée les 2 conteneurs qui vont recueillir ces infos
metascorewrap_note = []
metascorewrap_nbvote = []
count = 0

for i in scraping['metascorewrap']:
    metascorewrap = r.findall(i)
    if len(metascorewrap)>1 :
        #print(count," : ",metascorewrap)
        #print(type(metascorewrap_note))
        metascorewrap_note.append(metascorewrap[0])
        metascorewrap_nbvote.append(metascorewrap[1])
    else :
        metascorewrap_note.append(np.nan)
        metascorewrap_nbvote.append(np.nan)
    count+=1
    
# On recherche les floats dans Userscore correspond respectivement à la note et au nombre de vote
r = re.compile(r"[0-9]+\.?[0-9]+")

# On crée les 2 conteneurs qui vont recueillir ces infos
userscorewrap_note = []
userscorewrap_nbvote = []
count = 0
scraping['metaNote']=metascorewrap_note
scraping['metaNbVote']=metascorewrap_nbvote

for i in scraping['userscorewrap']:
    userscorewrap = r.findall(i)
    if len(userscorewrap)>1 :
        #print(count," : ",userscorewrap)
        #print(type(metascorewrap_note))
        userscorewrap_note.append(userscorewrap[0])
        userscorewrap_nbvote.append(userscorewrap[1])
    else :
        userscorewrap_note.append(np.nan)
        userscorewrap_nbvote.append(np.nan)
    count+=1
scraping['userNote']=userscorewrap_note
scraping['userNbVote']=userscorewrap_nbvote
    
# Comparatif entre le nom du dataset et le nom sur Metacritic
samename = []
for i,j in zip(scraping['Name'],scraping['gamename']):
    if (i in j) or (j in i) : # Je fais le test dans les 2 sens, voir index 16595
        samename.append(1)
    else :
        samename.append(0)
scraping['samename']=samename

In [55]:
#print(scraping['metascorewrap'][52])
#print(scraping['userscorewrap'][0])

scraping=scraping.drop(['metascorewrap','developper','userscorewrap','gamename','nbvotes','description','top','nbratings','metascore','ratings'],axis=1)
scraping

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,metaNote,metaNbVote,userNote,userNbVote,samename
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,76,51,8.1,453,1
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,84,14,8.5,98,1
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,82,73,8.4,1047,1
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,80,73,8.2,293,1
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,67,17,8.3,159,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,0
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01,50,19,NaN,NaN,1
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01,29,4,NaN,NaN,1
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,0


In [72]:
# Test Selenium

# Install chrome driver et selenium
#!pip install selenium
import time

#Importing packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# In the directory of chromedriver execute that in the console (for mac)
#xattr -d com.apple.quarantine chromedriver

# Download the chromedriver : be careful of the version of Chrome and your device (Mac M1 for exemple)
# https://chromedriver.chromium.org/downloads

for i,j in zip(vg['Name'],vg['Platform']):
    #i='wii sports'

#driver = webdriver.Chrome(executable_path='chromedriver')
    driver = webdriver.Chrome(executable_path='./chromedriver')

#driver = webdriver.Chrome(executable_path='/Users/pascal.thai/.Trash/chromedriver')

# Launch Google
    driver.get("https://google.com/")

# Click on the "J'accepte" button to accept cookies
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,"L2AGLb"))).click()

# Do the search on keyword Mario
    search = driver.find_element_by_name("q")
    search.send_keys(i," metacritic.com/game ")
    search.send_keys(Keys.RETURN)

# Recherche du premier lien avec l'intitulé Metacritic et clique dessus
    driver.find_element_by_partial_link_text('Metacritic').click()

# Acceptation des cookies sur la page metacritic
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,"onetrust-accept-btn-handler"))).click()

    data = {'Name':[],'Annee_sortie':[],'Developer':[],'Nombre_vote':[],'User_Score':[],"Nombre_avis":[],'Metascore':[]}
# On récupère le score utilisateur via la classe large
    data['User_Score'].append(driver.find_element_by_class_name("metascore_anchor").text)

# On récupère le nombre de vote utilisateur grâce au texte du lien "Ratings" (résultat à nettoyer)
    data['Nombre_avis'].append(driver.find_element_by_partial_link_text("Ratings").text)

# On récupère le score metacritic grâce à la classe xlarge
    data['Metascore'].append(driver.find_element_by_class_name("metascore_anchor").text)

# On récupère le nombre de note meta grâce à la classe count (résultat à nettoyer)
    data['Nombre_vote'].append(driver.find_element_by_class_name("count").text)

# Close the browser
    driver.quit()
    time.sleep(10)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Ratings"}
  (Session info: chrome=94.0.4606.71)


In [62]:
print("La note des joueurs est de ",data['user'])
print("Le metascore est de ",data['meta'])
print("Le nom du jeu est ",data['jeu'])

La note des joueurs est de  ['User Score\n6.2\nMixed or average reviews\nbased on 1655 Ratings']
Le metascore est de  ["Metascore\n79\nGenerally favorable reviews\nbased on 84 Critic Reviews\nWhat's this?"]
Le nom du jeu est  ["POKEMON: LET'S GO, PIKACHU! Switch"]


In [76]:
len(data['user'])

43

In [75]:
# Test Selenium

# Install chrome driver et selenium
#!pip install selenium
import time

#Importing packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

data = {'user':[],'meta':[],'jeu':[]}

# In the directory of chromedriver execute that in the console (for mac)
#xattr -d com.apple.quarantine chromedriver

# Download the chromedriver : be careful of the version of Chrome and your device (Mac M1 for exemple)
# https://chromedriver.chromium.org/downloads

# Ouverture de Chrome
driver = webdriver.Chrome(executable_path='./chromedriver')
init = 0

for i,j in zip(vg['Name'],vg['Platform']):
#i='wii sports'


    # Launch Google
    driver.get("https://google.com/")

    # Click on the "J'accepte" button to accept cookies
    if init == 0 :
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,"L2AGLb"))).click()

    # Do the search on keyword Mario
    search = driver.find_element_by_name("q")
    search.send_keys(i," metacritic.com/game ")
    search.send_keys(Keys.RETURN)

    # Recherche du premier lien avec l'intitulé Metacritic et clique dessus
    driver.find_element_by_partial_link_text('Metacritic').click()

    # Acceptation des cookies sur la page metacritic
    if init == 0 :
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,"onetrust-accept-btn-handler"))).click()

    
    # On récupère le score utilisateur via la classe userscore_wrap
    if (driver.find_element_by_class_name("userscore_wrap")):
        data['user'].append(driver.find_element_by_class_name("userscore_wrap").text)
    else :
        data['user'].append(np.nan)

    # On récupère le score metacritic grâce à la classe metascore_wrap
    if (driver.find_element_by_class_name("metascore_wrap")) :
        data['meta'].append(driver.find_element_by_class_name("metascore_wrap").text)
    else :
        data['meta'].append(np.nan)

    # On récupère le nom du jeu avec 
    if (driver.find_element_by_class_name("product_title")):
        data['jeu'].append(driver.find_element_by_class_name("product_title").text)
    else :
        data['jeu'].append(i)

    init += 1
    time.sleep(10)
    
# Close the browser
driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Metacritic"}
  (Session info: chrome=94.0.4606.71)
